In [1]:
# Calcula la exposición al cambio climático para grupos demográficos femeninos

# Importamos librerías
import os
import numpy as np
import pandas as pd
import xarray as xr
import geopandas as gpd
import rioxarray as rio
import geoviews as gv
import xesmf as xe

# Parámetros de mapas
gv.extension("matplotlib")
# Coloca la barra de color horizontal y abajo
def hook(plot, element):
    cax = plot.handles["cax"]
    ax = plot.handles["axis"]
    bbox = ax.get_position()
    l, b, w, h = bbox.x0, bbox.y0, bbox.width, bbox.height
    cax.set_position([l, 0.9*b, w, 0.05*h])
options = { "hooks": [hook], "ylim":(-62,85), "xlim":(-180,180),
    "colorbar": True,  "colorbar_opts": {"orientation": "horizontal"} } 
options_r = { "colorbar": True, "linewidth": 0.4, "hooks": [hook],
    "colorbar_opts": {"orientation": "horizontal"},
    "cmap": "plasma_r", "ylim":(-62,85), "xlim":(-180,180) }
options_m = { "bgcolor": "lightgray", "fontscale": 2,
    "aspect": 2.25, "ylim":(-62,85), "xlim":(-180,180) }

# Parámetros de visualización de tablas
pd.options.display.float_format = '{:,.1f}'.format

<img src='data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAEAAAABACAYAAACqaXHeAAAABHNCSVQICAgIfAhkiAAAAAlwSFlz
AAAB+wAAAfsBxc2miwAAABl0RVh0U29mdHdhcmUAd3d3Lmlua3NjYXBlLm9yZ5vuPBoAAA6zSURB
VHic7ZtpeFRVmsf/5966taWqUlUJ2UioBBJiIBAwCZtog9IOgjqACsogKtqirT2ttt069nQ/zDzt
tI4+CrJIREFaFgWhBXpUNhHZQoKBkIUASchWla1S+3ar7r1nPkDaCAnZKoQP/D7mnPOe9/xy76n3
nFSAW9ziFoPFNED2LLK5wcyBDObkb8ZkxuaoSYlI6ZcOKq1eWFdedqNzGHQBk9RMEwFAASkk0Xw3
ETacDNi2vtvc7L0ROdw0AjoSotQVkKSvHQz/wRO1lScGModBFbDMaNRN1A4tUBCS3lk7BWhQkgpD
lG4852/+7DWr1R3uHAZVQDsbh6ZPN7CyxUrCzJMRouusj0ipRwD2uKm0Zn5d2dFwzX1TCGhnmdGo
G62Nna+isiUqhkzuKrkQaJlPEv5mFl2fvGg2t/VnzkEV8F5ioioOEWkLG86fvbpthynjdhXYZziQ
x1hC9J2NFyi8vCTt91Fh04KGip0AaG9zuCk2wQCVyoNU3Hjezee9bq92duzzTmxsRJoy+jEZZZYo
GTKJ6SJngdJqAfRzpze0+jHreUtPc7gpBLQnIYK6BYp/uGhw9YK688eu7v95ysgshcg9qSLMo3JC
4jqLKQFBgdKDPoQ+Pltb8dUyQLpeDjeVgI6EgLIQFT5tEl3rn2losHVsexbZ3EyT9wE1uGdkIPcy
BGxn8QUq1QrA5nqW5i2tLqvrrM9NK6AdkVIvL9E9bZL/oyfMVd/jqvc8LylzRBKDJSzIExwhQzuL
QYGQj4rHfFTc8mUdu3E7yoLtbTe9gI4EqVgVkug2i5+uXGo919ixbRog+3fTbQ8qJe4ZOYNfMoTI
OoshUNosgO60AisX15aeI2PSIp5KiFLI9ubb1vV3Qb2ltwLakUCDAkWX7/nHKRmmGIl9VgYsUhJm
2NXjKYADtM1ygne9QQDIXlk49FBstMKx66D1v4+XuQr7vqTe0VcBHQlRWiOCbmmSYe2SqtL6q5rJ
zsTb7lKx3FKOYC4DoqyS/B5bvLPxvD9Qtf6saxYLQGJErmDOdOMr/zo96km1nElr8bmPOBwI9COv
HnFPRIwmkSOv9kcAS4heRsidOkpeWBgZM+UBrTFAXNYL5Vf2ii9c1trNzpYdaoVil3WIc+wdk+gQ
noie3ecCcxt9ITcLAPWt/laGEO/9U6PmzZkenTtsSMQ8uYywJVW+grCstAvCIaAdArAsIWkRDDs/
KzLm2YcjY1Lv0UdW73HabE9n6V66cxSzfEmuJssTpKGVp+0vHq73FwL46eOjpMpbRAnNmJFrGJNu
Ukf9Yrz+3rghiumCKNXXWPhLYcjxGsIpoCMsIRoFITkW8AuyM8jC1+/QLx4bozCEJIq38+1rtpR6
V/yzb8eBlRb3fo5l783N0CWolAzJHaVNzkrTzlEp2bQ2q3TC5gn6wpnoQAmwSiGh2GitnTmVMc5O
UyfKWUKCIsU7+fZDKwqdT6DDpvkzAX4/+AMFjk0tDp5GRXLpQ2MUmhgDp5gxQT8+Y7hyPsMi8uxF
71H0oebujHALECjFKaW9Lm68n18wXp2kVzIcABytD5iXFzg+WVXkegpAsOOYziqo0OkK76GyquC3
ltZAzMhhqlSNmmWTE5T6e3IN05ITFLM4GdN0vtZ3ob8Jh1NAKXFbm5PtLU/eqTSlGjkNAJjdgn/N
aedXa0tdi7+t9G0FIF49rtMSEgAs1kDLkTPO7ebm4IUWeyh1bKomXqlgMG6kJmHcSM0clYLJ8XtR
1GTnbV3F6I5wCGikAb402npp1h1s7LQUZZSMIfALFOuL3UUrfnS8+rez7v9qcold5tilgHbO1fjK
9ubb17u9oshxzMiUBKXWqJNxd+fqb0tLVs4lILFnK71H0Ind7uiPgACVcFJlrb0tV6DzxqqTIhUM
CwDf1/rrVhTa33/3pGPxJYdQ2l2cbgVcQSosdx8uqnDtbGjh9SlDVSMNWhlnilfqZk42Th2ZpLpf
xrHec5e815zrr0dfBZSwzkZfqsv+1FS1KUknUwPARVvItfKUY+cn57yP7qv07UE3p8B2uhUwLk09
e0SCOrK+hbdYHYLjRIl71wWzv9jpEoeOHhGRrJAzyEyNiJuUqX0g2sBN5kGK6y2Blp5M3lsB9Qh4
y2Ja6x6+i0ucmKgwMATwhSjdUu49tKrQ/pvN5d53ml2CGwCmJipmKjgmyuaXzNeL2a0AkQ01Th5j
2DktO3Jyk8f9vcOBQHV94OK+fPumJmvQHxJoWkaKWq9Vs+yUsbq0zGT1I4RgeH2b5wef7+c7bl8F
eKgoHVVZa8ZPEORzR6sT1BzDUAD/d9F78e2Tzv99v8D+fLVTqAKAsbGamKey1Mt9Ann4eH3gTXTz
idWtAJ8PQWOk7NzSeQn/OTHDuEikVF1R4z8BQCy+6D1aWRfY0tTGG2OM8rRoPaeIj5ZHzJxszElN
VM8K8JS5WOfv8mzRnQAKoEhmt8gyPM4lU9SmBK1MCQBnW4KONT86v1hZ1PbwSXPw4JWussVjtH9Y
NCoiL9UoH/6PSu8jFrfY2t36erQHXLIEakMi1SydmzB31h3GGXFDFNPaK8Rme9B79Ixrd0WN+1ij
NRQ/doRmuFLBkHSTOm5GruG+pFjFdAmorG4IXH1Qua6ASniclfFtDYt+oUjKipPrCQB7QBQ2lrgP
fFzm+9XWUtcqJ3/5vDLDpJ79XHZk3u8nGZ42qlj1+ydtbxysCezrydp6ugmipNJ7WBPB5tydY0jP
HaVNzs3QzeE4ZpTbI+ZbnSFPbVOw9vsfnVvqWnirPyCNGD08IlqtYkh2hjZ5dErEQzoNm+6ykyOt
Lt5/PQEuSRRKo22VkydK+vvS1XEKlhCJAnsqvcVvH7f/ZU2R67eXbMEGAMiIV5oWZWiWvz5Fv2xG
sjqNJQRvn3Rs2lji/lNP19VjAQDgD7FHhujZB9OGqYxRkZxixgRDVlqS6uEOFaJUVu0rPFzctrnF
JqijImVp8dEKVWyUXDk92zAuMZ6bFwpBU1HrOw6AdhQgUooChb0+ItMbWJitSo5Ws3IAOGEOtL53
0vHZih9sC4vtofZ7Qu6523V/fmGcds1TY3V36pUsBwAbSlxnVh2xLfAD/IAIMDf7XYIkNmXfpp2l
18rkAJAy9HKFaIr/qULkeQQKy9zf1JgDB2uaeFNGijo5QsUyacNUUTOnGO42xSnv4oOwpDi1zYkc
efUc3I5Gk6PhyTuVKaOGyLUAYPGIoY9Pu/atL/L92+4q9wbflRJ2Trpm/jPjdBtfnqB/dIThcl8A
KG7hbRuKnb8qsQsVvVlTrwQAQMUlf3kwJI24Z4JhPMtcfng5GcH49GsrxJpGvvHIaeem2ma+KSjQ
lIwUdYyCY8j4dE1KzijNnIP2llF2wcXNnsoapw9XxsgYAl6k+KzUXbi2yP3KR2ecf6z3BFsBICdW
nvnIaG3eHybqX7vbpEqUMT+9OL4Qpe8VON7dXuFd39v19FoAABRVePbGGuXTszO0P7tu6lghUonE
llRdrhArLvmKdh9u29jcFiRRkfLUxBiFNiqSU9icoZQHo5mYBI1MBgBH6wMNb+U7Pnw337H4gi1Y
ciWs+uks3Z9fztUvfzxTm9Ne8XXkvQLHNytOOZeiD4e0PgkAIAYCYknKUNUDSXEKzdWNpnil7r4p
xqkjTarZMtk/K8TQ6Qve78qqvXurGwIJqcOUKfUWHsm8KGvxSP68YudXq4pcj39X49uOK2X142O0
Tz5/u/7TVybqH0rSya6ZBwD21/gubbrgWdDgEOx9W

In [2]:
# Datos

# Códigos nacionales
ix  = "ISO_A3"

# Carpetas
path_r = "../share/Indexes/"
wb_path = "../../Bases_de_datos/CCKP_NetCDF/"

# Catálogo de datos
path_catalog = "../../Bases_de_datos/Data_catalog.csv"
df_c = pd.read_csv(path_catalog)

# Tabla base
iso = "../../Bases_de_datos/Country_ISO_code.csv"
df_iso = pd.read_csv(iso).set_index("alpha-3")
df_iso.index.name = ix

# Nombres de índices
index_n = [
    "Climate change risk index",
    "Climate change exposure index",
    "Social vulnerability index, physical climate impacts",
    "Sea level rise exposure index",
    "Drought exposure index",
    "Extreme heat exposure index",
    "Extreme rainfall exposure index",
    "Hurricane exposure index",
    "Life expectancy at birth index", "log GNI per capita, PPP index",
    "Gender Development index index", "% rural population index",
    "% population below 15 or above 65 years old index" ]
climate = index_n[1:2] + index_n[3:8]
social = index_n[2:3] + index_n[8:13]

# Índices climáticos y socioeconómicos
df_c = pd.read_csv(path_r + "climate_index.csv", index_col = ix)
df_s = pd.read_csv(path_r + "Physical_vulnerability_index.csv", index_col = ix)
df_r = pd.read_csv(path_r + "climate_risk_index.csv", index_col = ix)
df_iso[climate] = df_c[climate]
df_iso[social] = df_s[social]
df_iso[index_n[0]] = df_r[index_n[0]]
df_iso = df_iso[ df_iso[index_n[1:3]].notnull().all(axis = 1) ]

# Mapas
borders_path = ( "../../Bases_de_datos/Mapas/"
    + "Natural_Earth/ne_50m_admin_0_countries_mod" )
borders = gpd.read_file(borders_path).drop(
    columns = [ix] ).set_index("ISO_A3_EH")
borders.index.name = ix
borders = borders[ borders["ISO_N3_EH"] != "-99" ]
borders = borders[ ~borders.index.duplicated() ]
df_iso["ISO_N3_EH"] = borders["ISO_N3_EH"].astype(int)

In [3]:
# Funciones a utilizar

# Mejora el formato de las tablas para su uso en documentos.
def display(vn, var_i = None, p = False, format = "{:.1f}",
    category = "", type = "category", add = 1):
    # variables
    # vn:       variable principal
    # var_i:    conjunto de variables secundarias, solo si type = "index"
    # p:        indica si la variable es positiva
    # format:   formato a usar
    # category: nombre de la categoría, solo si type = "category"
    # type:     tipo de tabla a crear
    #           "index":    Resumen de variables
    #           "category": Una variable principal
    # add:      indica si sumar o promediar las columnas
    #
    # regresa
    # disp:     Objeto Display de Pandas o un Dataframe
    #           de pandas basado en la tabla de entrada

    # Países sin datos
    no_d = df_iso[ df_iso[vn].isnull() ].shape[0]
    print( f"Countries without data: {no_d} countries" )

    # Escogemos los 5 países más altos y otros más para formar la tabla
    # Resumen de variables
    if   type == "index":
        disp = df_iso.loc[ df_iso[vn].notnull(),
            ["name", vn] + var_i ].sort_values(
            vn, ascending = p ).reset_index(drop = True).head(15).copy()
    # Una variable principal
    elif type == "category":
        disp = df_iso.loc[ df_iso[vn].notnull(), ["name", vn] ].sort_values(
            vn, ascending = p ).reset_index(drop = True).head(15).copy()
    c_list = list( disp[ ["name", vn]
        ].sort_values(vn, ascending = p).head(5)["name"].values )
    print(f"Most vulnerable countries: {', '.join(c_list)}")

    # Categorías geopolíticas y geográficas
    cats = [ "", "", "", "", "", "", "SIDS", "LDC", "LLDC",
        "Asia", "Europe", "Africa", "Oceania", "Americas" ]
    # Iteramos para cada categoría geopolítica
    for r, cat in enumerate(cats[:9]):
        if r in range(0, 6): pass
        else:
            disp.iloc[r, 0] = cat
            # Resumen de variables
            if   type == "index":
                # Sumamos todo
                if add == 1: 
                    disp.iloc[r, 1:] = df_iso.loc[
                        df_iso[cat], [vn] + var_i ].sum()
                # Sumamos la población, promediamos porcentajes
                elif add == 0:
                    disp.iloc[r, 1] = df_iso.loc[
                        df_iso[cat], [vn] ].sum()
                    disp.iloc[r, 2] = df_iso.loc[
                        df_iso[cat], var_i[0] ].mean()
                    disp.iloc[r, 3:] = df_iso.loc[
                        df_iso[cat], var_i[1:] ].sum()
                # Promediamos todo
                elif add == -1:
                    disp.iloc[r, 1:] = df_iso.loc[
                        df_iso[cat], [vn] + var_i ].mean()
            # Una variable principal
            elif type == "category":
                # Sumamos la columna
                if add == 1:
                    disp.iloc[r, 1] = df_iso.loc[df_iso[cat], vn].sum()
                # Promediamos la columna
                elif add == 0:
                    disp.iloc[r, 1] = df_iso.loc[df_iso[cat], vn].mean()
            c_list = list( df_iso.loc[df_iso[cat], ["name", vn]
                ].sort_values( vn, ascending = p
                ).head(5)["name"].values )
            print(f"Most vulnerable {cat}: {', '.join(c_list)}")

    # Iteramos para cada categoría geográfica
    for r, cat in enumerate(cats):
        if r in range(0, 9): pass
        else:
            disp.iloc[r, 0] = cat
            # Resumen de variables
            if   type == "index":
                # Sumamos todo
                if add == 1:
                    disp.iloc[r, 1:] = df_iso.loc[
                        df_iso["region"] == cat, [vn] + var_i ].sum()
                # Sumamos la población, promediamos porcentajes
                elif add == 0:
                    disp.iloc[r, 1] = df_iso.loc[
                        df_iso["region"] == cat, [vn] ].sum()
                    disp.iloc[r, 2] = df_iso.loc[
                        df_iso["region"] == cat, var_i[0] ].mean()
                    disp.iloc[r, 3:] = df_iso.loc[
                        df_iso["region"] == cat, var_i[1:] ].sum()
                # Promediamos todo
                if add == -1:
                    disp.iloc[r, 1:] = df_iso.loc[
                        df_iso["region"] == cat, [vn] + var_i ].mean()
            # Una variable principal
            elif type == "category":
                # Sumamos la columna
                if add == 1:
                    disp.iloc[r, 1] = df_iso.loc[
                        df_iso["region"] == cat, vn].sum()
                # Promediamos la columna
                elif add == 0: 
                    disp.iloc[r, 2] = df_iso.loc[
                        df_iso["region"] == cat, vn].mean()

    # Total mundial
    cat = "World"
    r = 14
    disp.iloc[r, 0] = cat
    # Resumen de variables
    if   type == "index":
        # Sumamos todo
        if add == 1:
            disp.iloc[r, 1:] = df_iso[ [vn] + var_i ].sum()
        # Sumamos la población, promediamos porcentajes
        elif add == 0:
            disp.iloc[r, 1] = df_iso[ [vn] ].sum()
            disp.iloc[r, 2] = df_iso[ var_i[0] ].mean()
            disp.iloc[r, 3:] = df_iso[ var_i[1:] ].sum()
        # Promediamos todo
        elif add == -1:
            disp.iloc[r, 1:] = df_iso[ [vn] + var_i ].mean()
    # Una variable principal
    elif type == "category":
        # Sumamos la columna
        if add == 1:
            disp.iloc[r, 1] = df_iso[vn].sum()
        # Promediamos la columna
        elif add == 0:
            disp.iloc[r, 2] = df_iso[vn].sum()
    
    # Renombramos columnas
    # Resumen de variables
    if   type == "index":
        cols = ["Name", vn] + var_i
    # Una variable principal
    elif type == "category":
        cols = [ "Name", f"{category}" ]
    disp.columns = cols

    # Damos formato
    # Resumen de variables
    if   type == "index":
        # Pasamos la tabla sin formato
        pass
        # Una variable principal
    elif type == "category":
        disp = disp.style.format( { cols[1]: format } )

    # Regresamos la tabla
    return disp

In [4]:
# Pasa todos los archivos a una carpeta
# Correr solo una vez
'''
path   = "/Users/rodrigo/Downloads/"
path_d = f"{path}1km_pregnancies_countries/"
path_r = f"{path}1km_pregnancies/"

countries = os.listdir(path_d)

for c in countries:
    if c == ".DS_Store": pass
    else:
        files = os.listdir(path_d + c)
        iso = files[2][0:4]
        if files[2] == "README.txt": iso = files[3][0:4]
        for f in files:
            if f == "README.txt":
                os.rename( f"{path_d}{c}/{f}", f"{path_r}{iso}{f}" )
            else:
                os.rename(f"{path_d}{c}/{f}", f"{path_r}{f}")
'''

'\npath   = "/Users/rodrigo/Downloads/"\npath_d = f"{path}1km_pregnancies_countries/"\npath_r = f"{path}1km_pregnancies/"\n\ncountries = os.listdir(path_d)\n\nfor c in countries:\n    if c == ".DS_Store": pass\n    else:\n        files = os.listdir(path_d + c)\n        iso = files[2][0:4]\n        if files[2] == "README.txt": iso = files[3][0:4]\n        for f in files:\n            if f == "README.txt":\n                os.rename( f"{path_d}{c}/{f}", f"{path_r}{iso}{f}" )\n            else:\n                os.rename(f"{path_d}{c}/{f}", f"{path_r}{f}")\n'

In [5]:
# Códigos nacionales
ix  = "ISO_A3"

# Carpetas
file_path = "../results/hotspots_1km/" 
data_path = "../../Bases_de_datos/Worldpop/"

m   = 2
a   = 0
g   = 0
g_f = ["f", "m"]
g_n = ["Female", "Male"]
a_f = [0, 1]
a_n = ["0-12 months", "1-5 years"]
# Datos
names     = ["Pregnancies", "wcba2015", "age_sex_structures"]
name_path = [ f"{data_path}{x}/" for x in  names ]
var_n     = [ "Pregnant", "Women of child bearing age",
    f"{g_n[a]} population, {a_n[a]} old" ]
name_n    = [ f" affected {x.lower()}" for x in var_n ]
file_n    = [ f"{x}.csv" for x in names ]
data_n    = [ "_pregs_pp_v2_2015.tif" ]

# Variables de población afectada
vars     = [ "Only extreme rainfall", "Only extreme heat",
             "Only drought", "Only strong hurricanes",
             "Extreme rainfall & heat", "Extreme rainfall & drought",
             "Extreme rainfall & hurricanes", "Extreme heat & drought",
             "Extreme heat & hurricanes", "Drought & strong hurricanes",
             "Extreme rainfall, heat, & drought",
             "Extreme rainfall, heat, & hurricanes",
             "Extreme rainfall, drought, & hurricanes",
             "Extreme heat, drought, & hurricanes",
             "Extreme rainfall, heat, drought, & hurricanes" ]
var_clim = [ "Extreme rainfall", "Extreme heat",
             "Drought", "Strong hurricanes" ]
var_ci   = [
    [ "Only extreme rainfall", "Extreme rainfall & heat",
      "Extreme rainfall & drought", "Extreme rainfall & hurricanes",
      "Extreme rainfall, heat, & drought",
      "Extreme rainfall, heat, & hurricanes",
      "Extreme rainfall, drought, & hurricanes",
      "Extreme rainfall, heat, drought, & hurricanes" ],
    [ "Only extreme heat", "Extreme rainfall & heat",
      "Extreme heat & drought", "Extreme heat & hurricanes",
      "Extreme rainfall, heat, & drought",
      "Extreme rainfall, heat, & hurricanes",
      "Extreme heat, drought, & hurricanes",
      "Extreme rainfall, heat, drought, & hurricanes" ],
    [ "Only drought", "Extreme rainfall & drought",
      "Extreme heat & drought", "Drought & strong hurricanes",
      "Extreme rainfall, heat, drought, & hurricanes",
      "Extreme rainfall, heat, & drought",
      "Extreme rainfall, drought, & hurricanes",
      "Extreme heat, drought, & hurricanes", ],
    [ "Only strong hurricanes", "Extreme rainfall & hurricanes",
      "Extreme heat & hurricanes", "Drought & strong hurricanes",
      "Extreme rainfall, heat, & hurricanes",
      "Extreme rainfall, drought, & hurricanes",
      "Extreme heat, drought, & hurricanes",
      "Extreme rainfall, heat, drought, & hurricanes" ] ]
var_tot  = "Extreme climate"
# Archivos de zonas afectadas
files = [ "pre", "temp", "drought", "hurr",
    "temp_pre", "pre_drought", "pre_hurr",
    "temp_drought", "temp_hurr", "hurr_drought",
    "temp_pre_drought", "temp_pre_hurr",
    "pre_hurr_drought", "temp_drought_hurr",
    "temp_pre_hurr_drought" ]
files = [ f"{x}_2040_2059_SSP245.tif" for x in files ]

name_p    = [   f"{v}{name_n[m]}" for v in vars     ]
name_pp   = [ f"% {v}{name_n[m]}" for v in vars     ]
name_c    = [   f"{v}{name_n[m]}" for v in var_clim ]
name_cp   = [ f"% {v}{name_n[m]}" for v in var_clim ]
name_t  =       f"{var_tot}{name_n[m]}"
name_tp =     f"% {var_tot}{name_n[m]}"

name_ci = []
for x in var_ci:
    name_ci.append( [ f"{v}{name_n[m]}" for v in x ] )

In [6]:
# Creamos la columna de datos  si no existe
if not os.path.exists( f"../share/Indexes/{file_n[m]}" ):
    # Tabla base
    iso = "../../Bases_de_datos/Country_ISO_code.csv"
    df_iso = pd.read_csv(iso).set_index("alpha-3")
    df_iso.index.name = ix
    df_iso = df_iso.rename(columns = {"country-code": "ISO_N3"})
    df_iso[var_n[m]] = np.nan
    df_iso[name_p] = np.nan
    df_iso[name_pp] = None
    df_iso[["name", "ISO_N3", "region", "sub-region", "OECD", "EU27", "BRICS+",
        "BRICS", "LDC", "SIDS", "LLDC"] + name_p + name_pp
        ].to_csv( f"../share/Indexes/{file_n[m]}" )
else:
    df_iso = pd.read_csv( f"../share/Indexes/{file_n[m]}", index_col = ix )

# Iteramos para cada categoría climática
for i, v in enumerate(vars):
    print(v)

    # Cargamos los datos climáticos
    clim = xr.open_dataset( f"{file_path}{files[i]}"
        ).isel(band = 0).drop_vars( ["band", "spatial_ref"] ).rename_dims(
        {"x": "lon", "y": "lat"} ).rename_vars( {"x": "lon", "y": "lat"} )
    countries = xr.open_dataset( f"{data_path}global_level0_1km_2000_2020.tif"
        ).isel(band = 0).drop_vars( ["band", "spatial_ref"] ).rename_dims(
        {"x": "lon", "y": "lat"} ).rename_vars( {"x": "lon", "y": "lat"} )
    clim["lat"] = countries["lat"]
    clim["lon"] = countries["lon"]
    
    # Iteramos para cada país
    for row in df_iso.itertuples():
        # Información demográfica, corremos si existe
        if   m == 0:
            path_c = f"{name_path[m]}{row.Index}{data_n[m]}"
        elif m == 1:
            path_c = f"{name_path[m]}{names[m]}_{row.Index}.tif"
        elif m == 2:
            path_c = f"{name_path[m]}global_{g_f[g]}_{a_f[a]}_2020_1km.tif"
        if os.path.exists(path_c):
            df_iso = pd.read_csv( f"../share/Indexes/{file_n[m]}",
                index_col = ix )

            # Corremos si no se ha calculado ya
            if np.isnan(df_iso.loc[row.Index, name_pp[i]]): 
                print(row.Index, end = "\r")
                # Cargamos la información demográfica
                gender = xr.open_dataset(path_c).isel(band = 0).drop_vars(
                    ["band", "spatial_ref"] ).rename_dims(
                    {"x": "lon", "y": "lat"} ).rename_vars(
                    {"x": "lon", "y": "lat"} )
                gender["lat"] = countries["lat"]
                gender["lon"] = countries["lon"]
                gender["band_data"] = gender["band_data"].T
                lat_min = min( gender["lat"].values[0],
                    gender["lat"].values[-1] )
                lat_max = max( gender["lat"].values[0],
                    gender["lat"].values[-1] )
                lon_min = min( gender["lon"].values[0],
                    gender["lon"].values[-1] )
                lon_max = max( gender["lon"].values[0],
                    gender["lon"].values[-1] )
                clim_c   =   clim.where(countries == row.ISO_N3, 0)
                gender_c = gender.where(countries == row.ISO_N3, 0)

                # Calculamos la población expuesta
                df_iso.loc[row.Index, var_n[m]] = (
                    ( gender_c["band_data"].sum() ).values + 0 )
                df_iso.loc[row.Index, name_p[i]] = ( (
                    ( clim_c["band_data"] * gender_c["band_data"] )
                    ).sum().values + 0 )
                
                if df_iso.loc[row.Index, var_n[m]] == 0:
                    df_iso.loc[row.Index, name_pp[i]] = 0
                else:
                    df_iso.loc[row.Index, name_pp[i]] = (
                        100 * df_iso.loc[row.Index, name_p[i]]
                        / df_iso.loc[row.Index, var_n[m]] )

                # Guardamos el archivo
                df_iso.to_csv( f"../share/Indexes/{file_n[m]}" )

# Resultados
df_iso = pd.read_csv( f"../share/Indexes/{file_n[m]}", index_col = ix )
df_iso[name_t] = df_iso[name_p].sum(axis = 1)
df_iso[name_tp] = ( 100 * df_iso[name_t] / df_iso[var_n[m]] )
for i, v in enumerate(name_c):
    df_iso[v]  = df_iso[name_ci[i]].sum(axis = 1)
    df_iso[name_cp[i]] = 100 * df_iso[v] / df_iso[var_n[m]]
df_iso.to_csv( f"../share/Indexes/{file_n[m]}" )

print(f"Number of counries: {df_iso.dropna().shape[0]}")

disp = display(name_t, var_i = [ name_tp ] + name_c,
    p = False, type = "index", add = 0)
cols = disp.columns
disp = disp.style.format( { cols[1]: "{:,.0f}", cols[2]: "{:.1f}",
    cols[3]: "{:,.0f}", cols[4]: "{:,.0f}",
    cols[5]: "{:,.0f}", cols[6]: "{:,.0f}" } )
disp

Only extreme rainfall
Only extreme heat


KeyboardInterrupt: 

In [6]:
i = 0
path_c = f"{name_path[m]}global_{g_f[g]}_{a_f[a]}_2020_1km.tif"
clim = xr.open_dataset( f"{file_path}{files[i]}"
    ).isel(band = 0).drop_vars( ["band", "spatial_ref"] ).rename_dims(
    {"x": "lon", "y": "lat"} ).rename_vars( {"x": "lon", "y": "lat"} )
countries = xr.open_dataset( f"{data_path}global_level0_1km_2000_2020.tif"
    ).isel(band = 0).drop_vars( ["band", "spatial_ref"] ).rename_dims(
    {"x": "lon", "y": "lat"} ).rename_vars( {"x": "lon", "y": "lat"} )
clim["lat"] = countries["lat"]
clim["lon"] = countries["lon"]
gender = xr.open_dataset(path_c).isel(band = 0).drop_vars(
    ["band", "spatial_ref"] ).rename_dims(
    {"x": "lon", "y": "lat"} ).rename_vars(
    {"x": "lon", "y": "lat"} )
gender["lat"] = countries["lat"]
gender["lon"] = countries["lon"]
gender["band_data"] = gender["band_data"].T

In [9]:
a = clim * gender
a["countries"] = countries["band_data"]
a

<xarray.Dataset> Size: 6GB
Dimensions:    (lon: 43200, lat: 18720)
Coordinates:
  * lon        (lon) float64 346kB -180.0 -180.0 -180.0 ... 180.0 180.0 180.0
  * lat        (lat) float64 150kB 84.0 84.0 83.99 ... -71.97 -71.98 -71.99
Data variables:
    band_data  (lat, lon) float32 3GB nan nan nan nan nan ... nan nan nan nan
    countries  (lat, lon) float32 3GB ...

In [7]:
lat_min = min( gender["lat"].values[0],
    gender["lat"].values[-1] )
lat_max = max( gender["lat"].values[0],
    gender["lat"].values[-1] )
lon_min = min( gender["lon"].values[0],
    gender["lon"].values[-1] )
lon_max = max( gender["lon"].values[0],
    gender["lon"].values[-1] )
a = gender.sel( { "lat": slice(lat_max, (lat_max+lat_min)/2), 
    "lon": slice(lon_min, (lon_min+lon_max)/2) } )
b = countries.sel( { "lat": slice(lat_max, (lat_max+lat_min)/2), 
    "lon": slice(lon_min, (lon_min+lon_max)/2) } )
a

<xarray.Dataset> Size: 809MB
Dimensions:    (lon: 21600, lat: 9360)
Coordinates:
  * lon        (lon) float64 173kB -180.0 -180.0 -180.0 ... -0.01375 -0.005417
  * lat        (lat) float64 75kB 84.0 84.0 83.99 83.98 ... 6.029 6.02 6.012
Data variables:
    band_data  (lon, lat) float32 809MB ...

In [8]:
c = a.to_dataframe().reset_index(drop = True)

In [9]:
d = b.to_dataframe().reset_index(drop = True)

In [12]:
c["ISO_N3"] = d["band_data"]
c = c.set_index("ISO_N3")

In [22]:
df_iso = df_iso.set_index("country-code")
df_iso.index.name = "ISO_N3"

In [23]:
df_iso["a"] = c.groupby("ISO_N3").sum()
df_iso

,name,alpha-2,iso_3166-2,region,sub-region,intermediate-region,region-code,sub-region-code,intermediate-region-code,OECD,...,Hurricane exposure index,"Social vulnerability index, physical climate impacts",Life expectancy at birth index,"log GNI per capita, PPP index",Gender Development index index,% rural population index,% population below 15 or above 65 years old index,Climate change risk index,ISO_N3_EH,a
ISO_N3,,,,,,,,,,,,,,,,,,,,,
4,Afghanistan,AF,ISO 3166-2:AF,Asia,Southern Asia,NaN,142.0,34.0,NaN,False,...,0.0,9.5,8.4,9.5,9.9,8.9,9.2,7.4,4,NaN
8,Albania,AL,ISO 3166-2:AL,Europe,Southern Europe,NaN,150.0,39.0,NaN,False,...,0.0,3.5,2.8,4.1,3.9,4.7,2.6,5.3,8,NaN
12,Algeria,DZ,ISO 3166-2:DZ,Africa,Northern Africa,NaN,2.0,15.0,NaN,False,...,0.0,5.2,2.7,5.4,8.5,3.2,6.4,5.6,12,"49,811.0"
20,Andorra,AD,ISO 3166-2:AD,Europe,Southern Europe,NaN,150.0,39.0,NaN,False,...,0.0,0.4,0.5,1.0,NaN,1.4,0.6,1.0,20,NaN
24,Angola,AO,ISO 3166-2:AO,Africa,Sub-Saharan Africa,Middle Africa,2.0,202.0,17.0,False,...,0.0,7.7,8.7,7.2,8.0,4.1,9.6,4.8,24,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
862,Venezuela (Bolivarian Republic of),VE,ISO 3166-2:VE,Americas,Latin America and the Caribbean,South America,19.0,419.0,5.0,False,...,1.0,3.9,5.6,6.8,1.5,1.1,5.2,4.8,862,"282,816.8"
704,Viet Nam,VN,ISO 3166-2:VN,Asia,South-eastern Asia,NaN,142.0,35.0,NaN,False,...,2.3,3.9,3.7,5.5,1.1,7.8,2.0,5.6,704,NaN
887,Yemen,YE,ISO 3166-2:YE,Asia,Western Asia,NaN,142.0,145.0,NaN,False,...,0.0,9.0,8.1,9.7,9.9,7.8,8.0,9.0,887,NaN


In [15]:
c = c.sort_index()

: 

In [11]:
d

band_data
lon    lat            
-180.0 84.0        NaN
       84.0        NaN
       84.0        NaN
       84.0        NaN
       84.0        NaN
...                ...
-0.0   6.0       288.0
       6.0       288.0
       6.0       288.0
       6.0       288.0
       6.0       288.0

[202176000 rows x 1 columns]

In [8]:
c = a.to_dataframe()
d = b.to_dataframe()

: 

In [13]:
c["ISO_N3"] = d["band_data"]

: 

In [ ]:
# Parámetros de visualización de tablas
pd.options.display.float_format = '{:,.0f}'.format
df_iso.loc[ df_iso[name_tp] >= 100, ["name", name_t, name_tp] + name_c
    ].sort_values(name_tp, ascending = False)

,name,Extreme climate affected pregnant,% Extreme climate affected pregnant,Extreme rainfall affected pregnant,Extreme heat affected pregnant,Drought affected pregnant,Strong hurricanes affected pregnant
ISO_A3,,,,,,,
MUS,Mauritius,"20,263",100,0,0,0,"20,263"
CYM,Cayman Islands,"1,276",100,0,0,0,"1,276"
OMN,Oman,"114,135",100,"114,135","91,579",0,0
AIA,Anguilla,366,100,0,0,0,366
KNA,Saint Kitts and Nevis,"1,546",100,0,0,0,"1,546"
JOR,Jordan,"290,217",100,"12,292","20,193","288,402",0
ATG,Antigua and Barbuda,"2,536",100,0,0,0,"2,536"
REU,Réunion,"18,801",100,0,0,0,"18,801"
GLP,Guadeloupe,"10,868",100,0,0,"10,828","10,868"


In [ ]:
df_iso.loc[(df_iso["region"] == "Asia") & (df_iso["% Extreme climate affected pregnant"].notnull()), "name"].values

array(['Afghanistan', 'Azerbaijan', 'Bangladesh', 'Cambodia', 'China',
       'India', 'Indonesia', 'Jordan',
       "Korea (Democratic People's Republic of)",
       "Lao People's Democratic Republic", 'Lebanon', 'Myanmar', 'Nepal',
       'Oman', 'Pakistan', 'Palestine, State of', 'Philippines',
       'Tajikistan', 'Timor-Leste', 'Uzbekistan', 'Viet Nam'],
      dtype=object)